In [1]:
import requests
import json
import warnings
import asyncio
import aiohttp

import pandas as pd
import numpy as np

from pathlib import Path
from datetime import date
from tqdm import tqdm

warnings.simplefilter(action=r'ignore', category=FutureWarning)

In [2]:
class interval:
    def __init__(self, lower, upper) -> None:
        self.__lower = lower
        self.__upper = upper

    def get_lower(self):
        return self.__lower
    
    def get_upper(self):
        return self.__upper

INFO_DICT = {
        r'stockCode': 2,
        r'closePrice': 3,
        r'prevClosePrice': 4,
        r'openPrice': 5,
        r'highest': 33,
        r'lowest': 34,
        r'turnOverRate': 38,
        r'ampRate': 43,
        r'tmCap': 44,
    }

    # thresholds used to filter data
THRE_DICT = {
        r'ampRate': interval(3, 6),        # ampRate: +3~+6%
        r'turnOverRate': interval(5, 10),  # turnOverRate: +5%~+10%
        r'tmCap': interval(50, 120),       # tradableMarketCap: 50~120
    }

In [7]:
# DEVELOPMENT CODE

class StockInfoSeaker:
    def __init__(self, infoDict, threDict, stockList: list) -> None:
        self.__infoDict = infoDict
        self.__therDict = threDict
        self.__stockList = stockList
        self.__data = []

        self.df = pd.DataFrame()

    async def fetch_stock_data(self, session, stockCode):
        url = f'http://ifzq.gtimg.cn/appstock/app/kline/mkline?param={stockCode},m1,,10'
        async with session.get(url, allow_redirects=False) as response:
            if 300 <= response.status < 400:
                print(f'Warning: Request for stock {stockCode} was redirected.')
                return None

            elif response.status == 403:
                print(f"Warning: Request for stock {stockCode} was blocked by a firewall.")
                return None
            
            elif response.status == 200:
                info = await response.text()

                if 'window.location.href="https://waf.tencent.com/501page.html' in info:
                    print(f"Warning: Request for stock {stockCode} was blocked by a firewall.")
                    return None

                infoDict = json.loads(info)

                rawData = infoDict['data'][stockCode]['qt'][stockCode]
                selectedData = [rawData[i] for i in list(self.__infoDict.values())]
                selectedData[0] = stockCode
                selectedData[1:] = list(map(float, selectedData[1:]))

                return selectedData
            return None

    async def seak_data(self):
        async with aiohttp.ClientSession() as session:
            tasks = []
            for stockCode in self.__stockList:
                tasks.append(self.fetch_stock_data(session, stockCode))
            results = await asyncio.gather(*tasks)
            
            self.__data = [res for res in results if res is not None]

    def filter_data(self):
        self.df = pd.DataFrame(self.__data, columns=list(self.__infoDict.keys()))
        for k, v in self.__therDict.items():
            expr = str(v.get_lower()) + r' <= ' + str(k) + r' <= ' + str(v.get_upper())
            self.df.query(expr=expr, inplace=True)

    def save_data(self, savedir):
        self.df.to_csv(savedir, index=False)


async def main():
    df = pd.read_csv(r'./stock_code.csv', header=None)
    stockCodeList = df[df.columns[0]].values.tolist()

    seaker = StockInfoSeaker(infoDict=INFO_DICT,
                             threDict=THRE_DICT,
                             stockList=stockCodeList)
    
    await seaker.seak_data()
    seaker.filter_data()

    seaker.save_data(r'./interest_code.csv')

# 运行异步主函数
await main()


In [32]:
print(combined_results[0])
print(type(combined_results[0]))

{"code":0,"msg":"","data":{"sz000001":{"qt":{"sz000001":["51","\u5e73\u5b89\u94f6\u884c","000001","10.08","10.07","10.06","878865","467875","410990","10.07","6594","10.06","11345","10.05","9290","10.04","2383","10.03","5143","10.08","875","10.09","1504","10.10","1696","10.11","1254","10.12","795","","20240906161406","0.01","0.10","10.21","10.05","10.08\/878865\/891507834","878865","89151","0.45","4.17","","10.21","10.05","1.59","1956.09","1956.12","0.47","11.08","9.06","0.95","28631","10.14","3.78","4.21","","","0.90","89150.7834","0.0000","0"," ","GP-A","16.25","-0.79","7.13","9.74","0.82","11.02","8.24","-3.72","0.40","-0.98","19405617528","19405918198","70.04","15.85","19405617528","","","-5.00","0.10","","CNY","0","","10.00","15554"],"market":["2024-09-08 08:46:25|HK_close_\u5df2\u4f11\u5e02|SH_close_\u5df2\u4f11\u5e02|SZ_close_\u5df2\u4f11\u5e02|US_close_\u5df2\u4f11\u5e02|SQ_close_\u5df2\u4f11\u5e02|DS_close_\u5df2\u4f11\u5e02|ZS_close_\u5df2\u4f11\u5e02|NEWSH_close_\u5df2\u4f11\

In [34]:
dictinfo = json.loads(combined_results[0])

In [38]:
# TEST UNIT
INFO_DICT = {
        r'stockCode': 2,
        r'closePrice': 3,
        r'prevClosePrice': 4,
        r'openPrice': 5,
        r'highest': 33,
        r'lowest': 34,
        r'turnOverRate': 38,
        r'ampRate': 43,
        r'tmCap': 44,
    }

stockCode = r'sz000001'  # test stock code
resp = dictinfo

# print(resp['data'])
for k, v in INFO_DICT.items():
    print(k, end=': ')
    print(resp['data'][stockCode]['qt'][stockCode][v])

stockCode: 000001
closePrice: 10.08
prevClosePrice: 10.07
openPrice: 10.06
highest: 10.21
lowest: 10.05
turnOverRate: 0.45
ampRate: 1.59
tmCap: 1956.09


In [40]:
stockCode = r'sz000001'  # test stock code
url = f'http://ifzq.gtimg.cn/appstock/app/kline/mkline?param={stockCode},m1,,10'
resp = json.loads(requests.get(url).content)

# print(resp['data'][stockCode]['qt'][stockCode])
for k, v in INFO_DICT.items():
    print(k, end=': ')
    print(resp['data'][stockCode]['qt'][stockCode][v])

stockCode: 000001
closePrice: 10.08
prevClosePrice: 10.07
openPrice: 10.06
highest: 10.21
lowest: 10.05
turnOverRate: 0.45
ampRate: 1.59
tmCap: 1956.09
